# <ins> imports

In [1]:
from langdetect import detect, LangDetectException
import pandas as pd
import torch
import numpy as np
import re
# Read data from files  
df  = pd.read_parquet('https://github.com/omriallouche/ydata_deep_learning_2021/raw/master/data/metrolyrics.parquet')


<ins> Keep only metal genre

In [2]:
# df = df[(df['genre'] == 'Metal') | (df['genre'] == 'Rock')]
df = df[df['genre'] == 'Pop']

<ins> Remove non-english songs

In [3]:
# Assuming your DataFrame is called 'df' and the song column is 'song'
# Create a new column 'language' with the detected language for each song name
def detect_language(song_name):
    try:
        return detect(song_name)
    except LangDetectException:
        return ''

df['language'] = df['song'].apply(lambda x: detect_language(x) if pd.notnull(x) else '')

# Filter out rows where the language is not English
df = df[df['language'] == 'en']

# Drop the 'language' column if you don't need it anymore
df = df.drop(['language'], axis=1)


<ins> Clean data

In [4]:
import re
from unidecode import unidecode

def clean_data(text):
    # Remove punctuations
    text = re.sub(r'[^\w\s\n]', '', text)

    # Remove digits
    text = re.sub(r'[0-9]', '', text)

    # Remove line breaks and tabs
    # text = text.replace('', ' ')
    text = text.replace('\t', ' ')

    # Replace multiple subsequent empty spaces with a single space
    text = re.sub(r'\s{2,}', ' ', text)
    
    # Remove non-English characters
    text = unidecode(text)

    # Lowercase text
    text = text.lower()

    return text


def clean_data(text):
    # text = re.sub('\n', ' ', text)
    text = re.sub('[^a-zA-Z\n ]', '', text)
    text = text.lower()
    text = re.sub(r'\s{2,}', ' ', text)
    return text





In [5]:
df['sent'] = df['lyrics'].apply(clean_data)

In [6]:
text = ''

for song in df['sent']:
    text += song

chars = tuple(set(text))

In [7]:
int2char = dict(enumerate(chars))

char2int = {ch: ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])

In [8]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [9]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        
        # The features
        x = arr[:, n:n+n_steps]
        
        # The targets, shifted by one
        y = np.zeros_like(x)
        
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

# <ins> The network!

In [10]:
from torch import nn
import torch.nn.functional as F

class CharRNN(nn.Module):   
    def __init__(self, tokens, n_steps=100, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # Creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## Define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## Define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## Define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
        
        # Initialize the weights
        self.init_weights()
      
    
    def forward(self, x, hc):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hc`. '''
        
        ## Get x, and the new hidden state (h, c) from the lstm
        x, (h, c) = self.lstm(x, hc)
        
        ## pass x through the dropout layer
        x = self.dropout(x)
        
        # Reshape to fit fully-connected layer expected shape
        x = x.reshape(-1, self.n_hidden)
        
        ## Put x through the fully-connected layer
        x = self.fc(x)
        
        # Return x and the hidden state (h, c)
        return x, (h, c)
    
    
    def predict(self, char, h=None, cuda=False, top_k=None):
        ''' Given a character, predict the next character.
        
            Returns the predicted character and the hidden state.
        '''
        if cuda:
            self.cuda()
        else:
            self.cpu()
        
        if h is None:
            h = self.init_hidden(1)
        
        x = np.array([[self.char2int[char]]])
        x = one_hot_encode(x, len(self.chars))
        
        inputs = torch.from_numpy(x)
        
        if cuda:
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=1).data
        
        if cuda:
            p = p.cpu()
        
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        
        char = np.random.choice(top_ch, p=p/p.sum())
            
        return self.int2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fully connected layer '''
        initrange = 0.1
        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_(),
                weight.new(self.n_layers, n_seqs, self.n_hidden).zero_())
        

# <ins> The train

In [11]:
def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.001, clip=5, val_frac=0.1, cuda=False, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        n_seqs: Number of mini-sequences per mini-batch, aka batch size
        n_steps: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        cuda: Train with CUDA on a GPU
        print_every: Number of steps for printing training and validation loss
    
    '''
    
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if cuda:
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    
    for e in range(epochs):
        
        h = net.init_hidden(n_seqs)
        
        for x, y in get_batches(data, n_seqs, n_steps):
            
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net.forward(inputs, h)
            
            loss = criterion(output, targets.view(n_seqs*n_steps).type(torch.cuda.LongTensor))

            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            opt.step()
            
            if counter % print_every == 0:
                
                # Get validation loss
                val_h = net.init_hidden(n_seqs)
                val_losses = []
                
                for x, y in get_batches(val_data, n_seqs, n_steps):
                    
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if cuda:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net.forward(inputs, val_h)
                    val_loss = criterion(output, targets.view(n_seqs*n_steps).type(torch.cuda.LongTensor))
                
                    val_losses.append(val_loss.item())
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

# <ins> Run here

In [29]:

# Initialize and print the network
net = CharRNN(chars, n_hidden=512, n_layers=1,drop_prob=0.5)
n_seqs, n_steps = 128, 20

train(net, encoded, epochs=10, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, cuda=True, print_every=1000)

Epoch: 1/10... Step: 1000... Loss: 1.6238... Val Loss: 1.7020
Epoch: 1/10... Step: 2000... Loss: 1.5110... Val Loss: 1.5541
Epoch: 2/10... Step: 3000... Loss: 1.4507... Val Loss: 1.4843
Epoch: 2/10... Step: 4000... Loss: 1.4239... Val Loss: 1.4382
Epoch: 3/10... Step: 5000... Loss: 1.3446... Val Loss: 1.4050
Epoch: 3/10... Step: 6000... Loss: 1.3529... Val Loss: 1.3794
Epoch: 4/10... Step: 7000... Loss: 1.3346... Val Loss: 1.3738
Epoch: 4/10... Step: 8000... Loss: 1.3367... Val Loss: 1.3536
Epoch: 5/10... Step: 9000... Loss: 1.2645... Val Loss: 1.3475
Epoch: 5/10... Step: 10000... Loss: 1.2480... Val Loss: 1.3403
Epoch: 6/10... Step: 11000... Loss: 1.2444... Val Loss: 1.3388
Epoch: 6/10... Step: 12000... Loss: 1.1649... Val Loss: 1.3234
Epoch: 7/10... Step: 13000... Loss: 1.2388... Val Loss: 1.3235
Epoch: 7/10... Step: 14000... Loss: 1.1117... Val Loss: 1.3192
Epoch: 8/10... Step: 15000... Loss: 1.2011... Val Loss: 1.3179
Epoch: 8/10... Step: 16000... Loss: 1.1822... Val Loss: 1.3123
E

In [19]:
# change the name, for saving multiple files
model_name = 'rnn_1_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [27]:
# Initialize a new CharRNN instance
net = CharRNN(chars, n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])

# Load the checkpoint
checkpoint = torch.load('rnn_1_epoch.net')

# Load the state dictionary into the new instance
net.load_state_dict(checkpoint['state_dict'])


n_seqs, n_steps = 128, 200

train(net, encoded, epochs=10, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, cuda=True, print_every=50)

Epoch: 1/10... Step: 50... Loss: 1.2032... Val Loss: 1.2619
Epoch: 1/10... Step: 100... Loss: 1.2037... Val Loss: 1.2530
Epoch: 1/10... Step: 150... Loss: 1.2058... Val Loss: 1.2497
Epoch: 1/10... Step: 200... Loss: 1.1686... Val Loss: 1.2505
Epoch: 2/10... Step: 250... Loss: 1.1363... Val Loss: 1.2495
Epoch: 2/10... Step: 300... Loss: 1.1337... Val Loss: 1.2446
Epoch: 2/10... Step: 350... Loss: 1.1417... Val Loss: 1.2453
Epoch: 2/10... Step: 400... Loss: 1.1101... Val Loss: 1.2484
Epoch: 3/10... Step: 450... Loss: 1.1085... Val Loss: 1.2477
Epoch: 3/10... Step: 500... Loss: 1.1326... Val Loss: 1.2479
Epoch: 3/10... Step: 550... Loss: 1.0844... Val Loss: 1.2466
Epoch: 3/10... Step: 600... Loss: 1.0589... Val Loss: 1.2492
Epoch: 4/10... Step: 650... Loss: 1.1199... Val Loss: 1.2500
Epoch: 4/10... Step: 700... Loss: 1.1211... Val Loss: 1.2468
Epoch: 4/10... Step: 750... Loss: 1.0737... Val Loss: 1.2459
Epoch: 4/10... Step: 800... Loss: 1.0912... Val Loss: 1.2476
Epoch: 5/10... Step: 850.

In [20]:
def sample(net, size, prime='balls', top_k=None, cuda=False):
        
    if cuda:
        net.cuda()
    else:
        net.cpu()

    net.eval()
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    
    h = net.init_hidden(1)
    
    for ch in prime:
        char, h = net.predict(ch, h, cuda=cuda, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        
        char, h = net.predict(chars[-1], h, cuda=cuda, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [30]:
print(sample(net, 500, prime='rap is about life and life is about hardship\n', top_k=5, cuda=True))

rap is about life and life is about hardship
if youre the only one
this is the distance
with me
and im gonna be this is all about you baby this time
and i see its together inside
the money in the same of you i still got the way were so lonely
im looking for the track in the dark
we stay and so long we got one to me
talking to the moon out on the wass on somebody
all ive been so fire
i want the truth we could de you told you the same
im true light is that fall
and it all been someone to me
if you shell feel a shell
so hand to hand on and bri
